In [43]:
import numpy as np
from os import listdir
from os.path import join
import torch

In [18]:
npz_files = [f for f in listdir('car_racing_data') if f.endswith('.npz')]
npz_files

['rollout_93.npz',
 'rollout_86.npz',
 'rollout_67.npz',
 'rollout_62.npz',
 'rollout_2.npz',
 'rollout_81.npz',
 'rollout_3.npz',
 'rollout_100.npz',
 'rollout_90.npz',
 'rollout_27.npz',
 'rollout_32.npz',
 'rollout_65.npz',
 'rollout_5.npz',
 'rollout_39.npz',
 'rollout_30.npz',
 'rollout_18.npz',
 'rollout_31.npz',
 'rollout_74.npz',
 'rollout_55.npz',
 'rollout_45.npz',
 'rollout_44.npz',
 'rollout_63.npz',
 'rollout_33.npz',
 'rollout_16.npz',
 'rollout_25.npz',
 'rollout_68.npz',
 'rollout_51.npz',
 'rollout_52.npz',
 'rollout_77.npz',
 'rollout_26.npz',
 'rollout_4.npz',
 'rollout_66.npz',
 'rollout_88.npz',
 'rollout_91.npz',
 'rollout_85.npz',
 'rollout_69.npz',
 'rollout_6.npz',
 'rollout_38.npz',
 'rollout_41.npz',
 'rollout_36.npz',
 'rollout_87.npz',
 'rollout_73.npz',
 'rollout_94.npz',
 'rollout_71.npz',
 'rollout_84.npz',
 'rollout_22.npz',
 'rollout_42.npz',
 'rollout_58.npz',
 'rollout_98.npz',
 'rollout_64.npz',
 'rollout_60.npz',
 'rollout_40.npz',
 'rollout_28.npz

In [39]:
rollout_data = np.load(join('car_racing_data', npz_files[0]), allow_pickle=True)
obs = rollout_data['obs']
actions = rollout_data['action']
obs.shape, actions.shape

((950, 64, 64, 3), (950, 3))

In [40]:
from models.V import VAE
vae = VAE.load_from_checkpoint('residual_vae_final.ckpt', latent_size=32).to('cuda')

In [49]:
obs_tensor = obs / 255.0
obs_tensor = torch.from_numpy(obs_tensor).permute(0,3,1,2).float().to('cuda')
obs_tensor.shape
with torch.no_grad():
    mu, logvar = vae.encode(obs_tensor)
    latent = vae.latent(mu, logvar).cpu().numpy()
latent.shape, actions.shape

((950, 32), (950, 3))

In [50]:
import numpy as np
combined = np.concatenate([latent, actions], axis=1)
combined.shape

(950, 35)

In [51]:
x = combined[:-1]
y = combined[1:]
x.shape, y.shape

((949, 35), (949, 35))